# Variables categóricas

Hay muchos datos no numéricos por ahí. Aquí se explica cómo usarlo para el aprendizaje automático.

En este tutorial, aprenderá qué es una variable categórica, junto con tres enfoques para manejar este tipo de datos.

### Introducción

Una variable categórica toma sólo un número limitado de valores.

Considere una encuesta que le pregunte con qué frecuencia desayuna y ofrezca cuatro opciones: "Nunca", "Rara vez", "La mayoría de los días" o "Todos los días". En este caso, los datos son categóricos, porque las respuestas caen en un conjunto fijo de categorías.
Si las personas respondieran a una encuesta sobre qué marca de automóvil poseen, las respuestas se clasificarían en categorías como "Honda", "Toyota" y "Ford". En este caso, los datos también son categóricos.
Recibirá un error si intenta conectar estas variables en la mayoría de los modelos de aprendizaje automático en Python sin preprocesarlas primero. En este tutorial, compararemos tres enfoques que puede utilizar para preparar sus datos categóricos.

# Tres enfoques
#### 1) Eliminar variables categóricas
El enfoque más sencillo para tratar con variables categóricas es simplemente eliminarlas del conjunto de datos. Este enfoque sólo funcionará bien si las columnas no contienen información útil.

#### 2) Codificación ordinal
La codificación ordinal asigna cada valor único a un número entero diferente.

Este enfoque supone un orden de las categorías: "Nunca" (0) < "Rara vez" (1) < "La mayoría de los días" (2) < "Todos los días" (3).

Esta suposición tiene sentido en este ejemplo, porque existe una clasificación indiscutible de las categorías. No todas las variables categóricas tienen un orden claro en los valores, pero nos referimos a aquellas que lo tienen como variables ordinales. Para los modelos basados en árboles (como árboles de decisión y bosques aleatorios), se puede esperar que la codificación ordinal funcione bien con variables ordinales.

#### 3) Codificación One-Hot
La codificación one-hot crea nuevas columnas que indican la presencia (o ausencia) de cada valor posible en los datos originales. Para entender esto, trabajaremos con un ejemplo.

En el conjunto de datos original, "Color" es una variable categórica con tres categorías: "Rojo", "Amarillo" y "Verde". La codificación one-hot correspondiente contiene una columna para cada valor posible y una fila para cada fila del conjunto de datos original. Siempre que el valor original fuera "Rojo", ponemos un 1 en la columna "Rojo"; si el valor original era "Amarillo", ponemos un 1 en la columna "Amarillo", y así sucesivamente.

A diferencia de la codificación ordinal, la codificación one-hot no supone un ordenamiento de las categorías. Por lo tanto, puede esperar que este enfoque funcione particularmente bien si no hay un orden claro en los datos categóricos (por ejemplo, "Rojo" no es ni más ni menos que "Amarillo"). Nos referimos a las variables categóricas sin una clasificación intrínseca como variables nominales.

La codificación one-hot generalmente no funciona bien si la variable categórica toma una gran cantidad de valores (es decir, generalmente no la usará para variables que toman más de 15 valores diferentes).

## Ejemplo

Como en el tutorial anterior, trabajaremos con el conjunto de datos de Melbourne Housing.

No nos centraremos en el paso de carga de datos. En cambio, puedes imaginar que estás en un punto en el que ya tienes los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Echamos un vistazo a los datos de entrenamiento con el método head() a continuación.

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


A continuación, obtenemos una lista de todas las variables categóricas en los datos de entrenamiento.

Hacemos esto verificando el tipo de datos (o tipo d) de cada columna. El tipo de objeto indica que una columna tiene texto (en teoría, hay otras cosas que podrían ser, pero eso no es importante para nuestros propósitos). Para este conjunto de datos, las columnas con texto indican variables categóricas.

In [2]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


### Definir función para medir la calidad de cada enfoque

Definimos una función score_dataset() para comparar los tres enfoques diferentes para tratar con variables categóricas. Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio. En general, queremos que el MAE sea lo más bajo posible.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Puntuación del método 1 (eliminar variables categóricas)
Soltamos las columnas de objetos con el método select_dtypes().

In [4]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
175703.48185157913


In [5]:
drop_X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
3264,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
9845,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10799,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


#### Puntuación del método 2 (codificación ordinal)
Scikit-learn tiene una clase OrdinalEncoder que se puede utilizar para obtener codificaciones ordinales. Recorremos las variables categóricas y aplicamos el codificador ordinal por separado a cada columna.

In [7]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
165936.40548390493


En la celda de código anterior, para cada columna, asignamos aleatoriamente cada valor único a un número entero diferente. Este es un enfoque común que es más sencillo que proporcionar etiquetas personalizadas; sin embargo, podemos esperar un aumento adicional en el rendimiento si proporcionamos etiquetas mejor informadas para todas las variables ordinales.

#### Puntuación del método 3 (codificación One-Hot)
Usamos la clase OneHotEncoder de scikit-learn para obtener codificaciones one-hot. Hay una serie de parámetros que se pueden utilizar para personalizar su comportamiento.

. Configuramos handle_unknown='ignore' para evitar errores cuando los datos de validación contienen clases que no están representadas en los datos de entrenamiento, y

. establecer sparse=False garantiza que las columnas codificadas se devuelvan como una matriz numerosa (en lugar de una matriz dispersa).

Para usar el codificador, proporcionamos solo las columnas categóricas que queremos que se codifiquen en caliente. Por ejemplo, para codificar los datos de entrenamiento, proporcionamos X_train[object_cols]. (object_cols en la celda de código siguiente es una lista de los nombres de columnas con datos categóricos, por lo que X_train[object_cols] contiene todos los datos categóricos en el conjunto de entrenamiento).

In [6]:
from sklearn.preprocessing import OneHotEncoder

# Aplicar codificador one-hot a cada columna con datos categóricos
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Eliminar columnas categóricas (se reemplazarán con codificación one-hot)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Agregue columnas codificadas en one-hot  a funciones numéricas
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Asegúrese de que todas las columnas tengan tipo de cadena
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

c:\Users\karlo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MAE from Approach 3 (One-Hot Encoding):
166089.4893009678


In [7]:
OH_X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount,0,...,6,7,8,9,10,11,12,13,14,15
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3264,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9845,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10799,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### ¿Qué enfoque es mejor?

En este caso, eliminar las columnas categóricas (Método 1) obtuvo el peor resultado, ya que obtuvo la puntuación MAE más alta. En cuanto a los otros dos enfoques, dado que los puntajes MAE devueltos tienen un valor tan cercano, no parece haber ningún beneficio significativo para uno sobre el otro.

En general, la codificación one-hot (método 3) suele tener el mejor rendimiento, y eliminar las columnas categóricas (método 1) suele tener el peor rendimiento, pero varía según el caso.

### Conclusión
El mundo está lleno de datos categóricos. ¡Serás un científico de datos mucho más eficaz si sabes cómo utilizar este tipo de datos común!

# EJERCICIOS

En este ejercicio, trabajará con datos del concurso de precios de vivienda para usuarios de Kaggle Learn.

Ejecute la siguiente celda de código sin cambios para cargar los conjuntos de entrenamiento y validación en X_train, X_valid, y_train e y_valid. El conjunto de prueba se carga en X_test.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
​
# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/test.csv', index_col='Id')
​
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)
​
# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)
​
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

Utilice la siguiente celda de código para imprimir las primeras cinco filas de datos.

In [ ]:
X_train.head()

Observe que el conjunto de datos contiene variables numéricas y categóricas. Deberá codificar los datos categóricos antes de entrenar un modelo.

Para comparar diferentes modelos, usará la misma función score_dataset() del tutorial. Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Paso 1: eliminar columnas con datos categóricos
Comenzará con el enfoque más sencillo. Utilice la celda de código siguiente para preprocesar los datos en X_train y X_valid para eliminar columnas con datos categóricos. Establezca los DataFrames preprocesados en drop_X_train y drop_X_valid, respectivamente.

In [ ]:
# Fill in the lines below: drop columns in training and validation data
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

Ejecute la siguiente celda de código para obtener el MAE para este enfoque.

In [ ]:
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

    MAE from Approach 1 (Drop categorical variables):
    17837.82570776256

Antes de pasar a la codificación ordinal, investigaremos el conjunto de datos. Específicamente, veremos la columna 'Condición2'. La siguiente celda de código imprime las entradas únicas tanto en el conjunto de entrenamiento como en el de validación.

### Paso 2: codificación ordinal
#### Parte A
Si ahora escribe código en:

. ajustar un codificador ordinal a los datos de entrenamiento, y luego

. Úselo para transformar los datos de entrenamiento y validación,

obtendrás un error. ¿Puedes ver por qué este es el caso? (Necesitará utilizar el resultado anterior para responder esta pregunta).

Ajustar un codificador ordinal a una columna de los datos de entrenamiento crea una etiqueta de valor entero correspondiente para cada valor único que aparece en los datos de entrenamiento. En el caso de que los datos de validación contengan valores que no aparecen también en los datos de entrenamiento, el codificador arrojará un error, porque estos valores no tendrán un número entero asignado. Observe que la columna 'Condición2' en los datos de validación contiene los valores 'RRAn' y 'RRNn', pero estos no aparecen en los datos de entrenamiento; por lo tanto, si intentamos usar un codificador ordinal con scikit-learn, el El código arrojará un error.

Este es un problema común que encontrará con los datos del mundo real y existen muchos enfoques para solucionarlo. Por ejemplo, puede escribir un codificador ordinal personalizado para manejar nuevas categorías. Sin embargo, el enfoque más sencillo es eliminar las columnas categóricas problemáticas.

Ejecute la celda de código siguiente para guardar las columnas problemáticas en una lista de Python bad_label_cols. Del mismo modo, las columnas que pueden codificarse de forma ordinal de forma segura se almacenan en good_label_cols.

In [ ]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

#### Parte B
Utilice la siguiente celda de código para codificar ordinalmente los datos en X_train y X_valid. Establezca los DataFrames preprocesados en label_X_train y label_X_valid, respectivamente.

Proporcionamos el código a continuación para eliminar las columnas categóricas en bad_label_cols del conjunto de datos.
Debe codificar de forma ordinal las columnas categóricas en good_label_cols.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply ordinal encoder 
ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])
# Your code here

Ejecute la siguiente celda de código para obtener el MAE para este enfoque.

In [ ]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

Hasta ahora, ha probado dos enfoques diferentes para tratar con variables categóricas. Y ha visto que codificar datos categóricos produce mejores resultados que eliminar columnas del conjunto de datos.

Pronto probará la codificación one-hot. Antes de eso, hay un tema adicional que debemos cubrir. Comience ejecutando la siguiente celda de código sin cambios.

In [ ]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

### Paso 3: Investigar la cardinalidad
#### Parte A
El resultado anterior muestra, para cada columna con datos categóricos, el número de valores únicos en la columna. Por ejemplo, la columna "Street" en los datos de entrenamiento tiene dos valores únicos: "Grvl" y "Pave", correspondientes a un camino de grava y un camino pavimentado, respectivamente.

Nos referimos al número de entradas únicas de una variable categórica como la cardinalidad de esa variable categórica. Por ejemplo, la variable 'Street' tiene cardinalidad 2.

Utilice el resultado anterior para responder las preguntas siguientes.

In [ ]:
# Fill in the line below: How many categorical variables in the training data
# have cardinality greater than 10?
high_cardinality_numcols = 3

# Fill in the line below: How many columns are needed to one-hot encode the 
# 'Neighborhood' variable in the training data?
num_cols_neighborhood = 25

#### Parte B
Para conjuntos de datos grandes con muchas filas, la codificación one-hot puede ampliar considerablemente el tamaño del conjunto de datos. Por esta razón, normalmente solo codificaremos columnas con una cardinalidad relativamente baja. Luego, las columnas de alta cardinalidad se pueden eliminar del conjunto de datos o podemos usar codificación ordinal.

Como ejemplo, considere un conjunto de datos con 10.000 filas y que contiene una columna categórica con 100 entradas únicas.

Si esta columna se reemplaza con la codificación one-hot correspondiente, ¿cuántas entradas se agregan al conjunto de datos?
Si reemplazamos la columna con la codificación ordinal, ¿cuántas entradas se agregan?
Utilice sus respuestas para completar las líneas a continuación.

In [ ]:
# Fill in the line below: How many entries are added to the dataset by 
# replacing the column with a one-hot encoding?
OH_entries_added = 1e4*100 - 1e4

# Fill in the line below: How many entries are added to the dataset by
# replacing the column with an ordinal encoding?
label_entries_added = 0

A continuación, experimentará con la codificación one-hot. Pero, en lugar de codificar todas las variables categóricas en el conjunto de datos, solo creará una codificación única para columnas con cardinalidad menor que 10.

Ejecute la siguiente celda de código sin cambios para configurar low_cardinality_cols en una lista de Python que contenga las columnas que se codificarán en caliente. Asimismo, high_cardinality_cols contiene una lista de columnas categóricas que se eliminarán del conjunto de datos.

In [ ]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

### Paso 4: codificación one-hot
Utilice la siguiente celda de código para codificar en caliente los datos en X_train y X_valid. Establezca los DataFrames preprocesados en OH_X_train y OH_X_valid, respectivamente.

La lista completa de columnas categóricas en el conjunto de datos se puede encontrar en la lista de Python object_cols.
Solo debe codificar una vez las columnas categóricas en low_cardinality_cols. Todas las demás columnas categóricas deben eliminarse del conjunto de datos.